In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
def load_Provider_charge_data(p):
    files = ['Outpatient_Data_2011_XLSX/Medicare_Provider_Charge_Outpatient_APC30_CY2011_v2.xlsx',
             'Outpatient_Data_2012_XLSX/Medicare_Provider_Charge_Outpatient_APC30_CY2012.xlsx',
             'Medicare_Provider_Charge_Outpatient_APC30_CY2013_v2_xlsx/Medicare_Provider_Charge_Outpatient_APC30_CY2013_v2.xlsx',
             'Medicare_Provider_Charge_Outpatient_APC32_CY2014.xlsx']
    df = pd.DataFrame([])
    for index,file in enumerate(files):
        db = pd.read_excel(p+file,skiprows=6,converters={'Provider ID':str})
        db['year'] = 2011+index
        df = df.append(db)
    df.rename(columns = {'Provider Id':'Provider ID'},inplace=True)
    return df

In [3]:
def load_APC_Provider_Data(p):
    files = ['MUP_OHP_R19_P04_V10_D15_APC_Provider.xlsx',
             'MUP_OHP_R19_P04_V10_D16_APC_Provider.xlsx',
             'MUP_OHP_R19_P04_V10_D17_APC_Provider.xlsx']
    df = pd.DataFrame([])
    cols = ['Average Estimated Total Submitted Charges', 'Average Medicare Allowed Amount', 
'Average Medicare Payment Amount', 'Outlier Comprehensive APC Services', 'Average Medicare Outlier Amount']
    for index,file in enumerate(files):
        db = pd.read_excel(p+file,skiprows=5,dtype=str)
        db['year'] = 2015+index
        db.columns=db.columns.str.replace('\n',' ') 
        df = df.append(db)
    for col in cols:
        df[col] = pd.to_numeric(df[col])
    return df

In [4]:
def load_detailed_dataset():
    db = pd.read_excel('../gtmsa_practicum_datasets/Detail_Data_new.xlsx',
                        sheet_name='Table 1. PF Hospitals',skiprows=9,dtype=str)
    db.columns=db.columns.str.replace('\n',' ')
    db.columns = db.columns.str.lower()
    df = pd.read_excel('../gtmsa_practicum_datasets/Detailed_Data.xlsx',
                        sheet_name='Table 1. Hospitals',skiprows=16,dtype=str)
    df.columns=df.columns.str.replace('\n',' ')
    df.columns = df.columns.str.lower()
    
    dt = pd.concat([db,df])
    dt = dt.drop_duplicates(subset=(['medicare provider number']),keep='first')
    print('Final detailed data shape :',dt.shape)
    return dt

In [5]:
def load_data():
    path = '../../../../gtmsa_practicum_datasets/OPPS_charge_data/'
    df = load_APC_Provider_Data(path)
    db = load_detailed_dataset()
    return df,db
opps_data, det_data = load_data()

Final detailed data shape : (3329, 30)


In [6]:
def convert_opps_grouped_data(dt):
    
    cols = ['Provider ID' ,'year','Beneficiaries',
'Comprehensive APC Services', 'Average Estimated Total Submitted Charges', 'Average Medicare Allowed Amount', 
'Average Medicare Payment Amount', 'Outlier Comprehensive APC Services', 'Average Medicare Outlier Amount']
    dt = dt[cols].groupby(by=['Provider ID' ,'year']).mean()
    dt = dt.unstack('year')
    dt['medicare provider number'] = dt.index
    return dt
data = convert_opps_grouped_data(opps_data)

In [7]:
data.to_csv('../gtmsa_practicum_datasets/OPPS_final_data.csv',index=False)